In [34]:
import yfinance as yf
import plotly.io as pio
import plotly.graph_objects as go

In [35]:
equity = yf.Ticker("SPY")
data = equity.history(period='1y')
df = data[['Close']]

sma = df.rolling(window=20).mean().dropna()
rstd = df.rolling(window=20).std().dropna()

upper_band = sma + 2 * rstd
lower_band = sma - 2 * rstd

upper_band = upper_band.rename(columns={'Close': 'upper'})
lower_band = lower_band.rename(columns={'Close': 'lower'})
bb = df.join(upper_band).join(lower_band)
bb = bb.dropna()

buyers = bb[bb['Close'] <= bb['lower']]
sellers = bb[bb['Close'] >= bb['upper']]

# Plotting

pio.templates.default = "plotly_dark"

fig = go.Figure()
fig.add_trace(go.Scatter(x=lower_band.index, 
                         y=lower_band['lower'], 
                         name='Lower Band', 
                         line_color='rgba(173,204,255,0.2)'
                        ))
fig.add_trace(go.Scatter(x=upper_band.index, 
                         y=upper_band['upper'], 
                         name='Upper Band', 
                         fill='tonexty', 
                         fillcolor='rgba(173,204,255,0.2)', 
                         line_color='rgba(173,204,255,0.2)'
                        ))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['Close'], 
                         name='Close', 
                         line_color='#636EFA'
                        ))
fig.add_trace(go.Scatter(x=sma.index, 
                         y=sma['Close'], 
                         name='SMA', 
                         line_color='#FECB52'
                        ))
fig.add_trace(go.Scatter(x=buyers.index, 
                         y=buyers['Close'], 
                         name='Buyers', 
                         mode='markers',
                         marker=dict(
                             color='#00CC96',
                             size=10,
                             )
                         ))
fig.add_trace(go.Scatter(x=sellers.index, 
                         y=sellers['Close'], 
                         name='Sellers', 
                         mode='markers', 
                         marker=dict(
                             color='#EF553B',
                             size=10,
                             )
                         ))
fig.show()

In [36]:
import plotly.express as px
ticker = yf.Ticker("aapl")
data = ticker.history(period="1y")
data = data.reset_index()
data['moving_average'] = data['Close'].rolling(window = 20).mean()
data['moving_average']
data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,moving_average
0,2021-10-01,141.109282,142.123602,138.334835,141.855103,94639600,0.0,0,NaN
1,2021-10-04,140.970050,141.417555,137.499507,138.364655,98322000,0.0,0,NaN
2,2021-10-05,138.712717,141.447393,138.583436,140.323685,80861100,0.0,0,NaN
3,2021-10-06,138.692824,141.357883,137.598948,141.208725,83221100,0.0,0,NaN
4,2021-10-07,142.262832,143.416372,141.924731,142.491547,61732700,0.0,0,NaN
...,...,...,...,...,...,...,...,...,...
247,2022-09-26,149.660004,153.770004,149.639999,150.770004,93339400,0.0,0,155.414500
248,2022-09-27,152.740005,154.720001,149.949997,151.759995,84442700,0.0,0,154.933499
249,2022-09-28,147.639999,150.639999,144.839996,149.839996,146691400,0.0,0,154.479999
250,2022-09-29,146.100006,146.720001,140.679993,142.479996,128138200,0.0,0,153.742999


In [37]:
import pandas as pd

In [38]:
def pandas_rsi(df: pd.DataFrame, window_length: int = 14, output: str = None, price: str = 'Close'):
    """
    An implementation of Wells Wilder's RSI calculation as outlined in
    his 1978 book "New Concepts in Technical Trading Systems" which makes
    use of the α-1 Wilder Smoothing Method of calculating the average
    gains and losses across trading periods and the Pandas library.
    @author: https://github.com/alphazwest
    Args:
        df: pandas.DataFrame - a Pandas Dataframe object
        window_length: int - the period over which the RSI is calculated. Default is 14
        output: str or None - optional output path to save data as CSV
        price: str - the column name from which the RSI values are calcuated. Default is 'Close'
    Returns:
        DataFrame object with columns as such, where xxx denotes an inconsequential
        name of the provided first column:
            ['xxx', 'diff', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rs', 'rsi']
    """
    # Calculate Price Differences using the column specified as price.
    df['diff1'] = df[price].diff(1)

    # Calculate Avg. Gains/Losses
    df['gain'] = df['diff1'].clip(lower=0).round(2)
    df['loss'] = df['diff1'].clip(upper=0).abs().round(2)

    # Get initial Averages
    df['avg_gain'] = df['gain'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]
    df['avg_loss'] = df['loss'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]

    # Calculate Average Gains
    for i, row in enumerate(df['avg_gain'].iloc[window_length+1:]):
        df['avg_gain'].iloc[i + window_length + 1] =\
            (df['avg_gain'].iloc[i + window_length] *
             (window_length - 1) +
             df['gain'].iloc[i + window_length + 1])\
            / window_length

    # Calculate Average Losses
    for i, row in enumerate(df['avg_loss'].iloc[window_length+1:]):
        df['avg_loss'].iloc[i + window_length + 1] =\
            (df['avg_loss'].iloc[i + window_length] *
             (window_length - 1) +
             df['loss'].iloc[i + window_length + 1])\
            / window_length

    # Calculate RS Values
    df['rs'] = df['avg_gain'] / df['avg_loss']

    # Calculate RSI
    df['rsi'] = 100 - (100 / (1.0 + df['rs']))

    # Save if specified
    if output is not None:
        df.to_csv(output)

    return df

In [40]:

from dash import Dash, html, dcc, Input, Output, callback
import dash
import plotly.express as px
import pandas as pd
import yfinance as yf
import quandl
import plotly.io as pio
import numpy as np
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import datetime as dt
from pandas_datareader import data as web
from dateutil.relativedelta import relativedelta
from plotly.subplots import make_subplots


In [47]:
# Import necessary libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import yfinance as yf
pio.templates.default = "plotly"

# Download historic pricing data for $BTC-USD
data = yf.Ticker('aapl').history(start='2021-01-01', end='2021-07-01')[['Open', 'High', 'Low', 'Close', 'Volume']]
# Force lowercase column names
data.columns = map(str.lower, data.columns)

# Make RSI Calculations
pandas_rsi(df=data, window_length=14, price='close')

# Set option to view all columns
pd.set_option('display.max_columns', None)
# Create Figure
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_width=[0.25, 0.75])

# Create Candlestick chart for price data
fig.add_trace(go.Candlestick(
    x=data.index,
    open=data['open'],
    high=data['high'],
    low=data['low'],
    close=data['close'],
    increasing_line_color='#ff9900',
    decreasing_line_color='black',
    showlegend=False
), row=1, col=1)

# Make RSI Plot
fig.add_trace(go.Scatter(
    x=data.index,
    y=data['rsi'],
    line=dict(color='#ff9900', width=2),
    showlegend=False,
), row=2, col=1
)

# Add upper/lower bounds
fig.update_yaxes(range=[-10, 110], row=2, col=1)
fig.add_hline(y=0, col=1, row=2, line_color="#666", line_width=2)
fig.add_hline(y=100, col=1, row=2, line_color="#666", line_width=2)

# Add overbought/oversold
fig.add_hline(y=30, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')
fig.add_hline(y=70, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')

# Customize font, colors, hide range slider
layout = go.Layout(
    plot_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    )
)

# update and display
fig.update_layout(layout)
fig.show()

C:\Users\mnomo\AppData\Local\Temp\ipykernel_9868\1711622038.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\mnomo\AppData\Local\Temp\ipykernel_9868\1711622038.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

